In [2]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-07-30 09:16:35,757 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-skip.model')
model.init_sims(replace=True)

2017-07-30 09:16:38,305 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-skip.model
2017-07-30 09:16:44,871 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-skip.model.wv.* with mmap=None
2017-07-30 09:16:44,872 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-skip.model.wv.syn0.npy with mmap=None
2017-07-30 09:16:52,751 : INFO : setting ignored attribute syn0norm to None
2017-07-30 09:16:52,752 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-skip.model.syn1neg.npy with mmap=None
2017-07-30 09:17:41,008 : INFO : setting ignored attribute cum_table to None
2017-07-30 09:17:41,809 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-skip.model
2017-07-30 09:19:24,145 : INFO : precomputing L2-norms of word weight vectors


In [4]:
num_features = model.wv.syn0.shape[1]

In [6]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
df.shape

(1039, 186)

In [7]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [8]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['extraversion_m']
    evaluate_prediction(predictions, target)

In [9]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if (word in index2word_set): 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords < 0:
        print 'sorry, empty...'
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
        #Print a status message every 1000th review
        if counter%100. == 0.:
            print "Review %d of %d" % (counter, len(reviews))
        #Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, model.wv.syn0.shape[1])
        #Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [10]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            word = word.lower()
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [11]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [12]:
trainDataVecs = getAvgFeatureVecs( train_tokenized, model, num_features )
testDataVecs = getAvgFeatureVecs( test_tokenized, model, num_features )

Review 0 of 727
Review 100 of 727
Review 200 of 727
Review 300 of 727
Review 400 of 727
Review 500 of 727
Review 600 of 727
Review 700 of 727
Review 0 of 312
Review 100 of 312
Review 200 of 312
Review 300 of 312


In [13]:
del model

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

## Extraversion evaluation

In [15]:
df.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [16]:
%%time
model_trainer.fit(trainDataVecs, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   25.3s finished


CPU times: user 5.23 s, sys: 381 ms, total: 5.61 s
Wall time: 32.8 s


In [17]:
yp = model.predict(testDataVecs)
yt = test_w2v_data['extraversion_m']

In [18]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,93,49
1,79,91


In [19]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.54      0.65      0.59       142
          1       0.65      0.54      0.59       170

avg / total       0.60      0.59      0.59       312



In [20]:
accuracy_score(yt, yp)

0.58974358974358976

In [21]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [22]:
ytp = model.predict(trainDataVecs)
ytt = train_w2v_data['extraversion_m']

In [23]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       387
          1       1.00      1.00      1.00       340

avg / total       1.00      1.00      1.00       727



In [24]:
print accuracy_score(ytt, ytp)

0.998624484182


## Agreableness evaluation

In [25]:
df.agreeabeness_m.value_counts()

1    542
0    497
Name: agreeabeness_m, dtype: int64

In [26]:
%%time
model_trainer.fit(trainDataVecs, train_w2v_data['agreeabeness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   26.7s finished


CPU times: user 5.85 s, sys: 81.1 ms, total: 5.93 s
Wall time: 32.6 s


In [27]:
yp = model.predict(testDataVecs)
yt = test_w2v_data['agreeabeness_m']

In [28]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,73,68
1,72,99


In [29]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.50      0.52      0.51       141
          1       0.59      0.58      0.59       171

avg / total       0.55      0.55      0.55       312



In [30]:
accuracy_score(yt, yp)

0.55128205128205132

In [31]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [32]:
ytp = model.predict(trainDataVecs)
ytt = train_w2v_data['agreeabeness_m']

In [33]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       356
          1       1.00      1.00      1.00       371

avg / total       1.00      1.00      1.00       727



In [34]:
print accuracy_score(ytt, ytp)

1.0


## Conscientiousness evaluation

In [35]:
df.conscientiousness_m.value_counts()

0    523
1    516
Name: conscientiousness_m, dtype: int64

In [36]:
%%time
model_trainer.fit(trainDataVecs, train_w2v_data['conscientiousness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   28.0s finished


CPU times: user 5.64 s, sys: 89.1 ms, total: 5.73 s
Wall time: 33.7 s


In [37]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,73,68
1,72,99


In [38]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.50      0.52      0.51       141
          1       0.59      0.58      0.59       171

avg / total       0.55      0.55      0.55       312



In [39]:
accuracy_score(yt, yp)

0.55128205128205132

In [40]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [41]:
ytp = model.predict(trainDataVecs)
ytt = train_w2v_data['conscientiousness_m']

In [42]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       356
          1       0.99      1.00      1.00       371

avg / total       1.00      1.00      1.00       727



In [43]:
print accuracy_score(ytt, ytp)

0.997248968363


## Neuroticism evation

In [44]:
df.neuroticism_m.value_counts()

0    535
1    504
Name: neuroticism_m, dtype: int64

In [45]:
%%time
model_trainer.fit(trainDataVecs, train_w2v_data['neuroticism_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   26.8s finished


CPU times: user 5.52 s, sys: 95.5 ms, total: 5.61 s
Wall time: 32.4 s


In [46]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,73,68
1,72,99


In [47]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.50      0.52      0.51       141
          1       0.59      0.58      0.59       171

avg / total       0.55      0.55      0.55       312



In [48]:
accuracy_score(yt, yp)

0.55128205128205132

In [49]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [50]:
ytp = model.predict(trainDataVecs)
ytt = train_w2v_data['neuroticism_m']

In [51]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       372
          1       0.99      1.00      1.00       355

avg / total       1.00      1.00      1.00       727



In [52]:
print accuracy_score(ytt, ytp)

0.997248968363


## Openess evaluation

In [53]:
df.openness_m.value_counts()

1    543
0    496
Name: openness_m, dtype: int64

In [54]:
%%time
model_trainer.fit(trainDataVecs, train_w2v_data['openness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   27.5s finished


CPU times: user 5.4 s, sys: 85.8 ms, total: 5.49 s
Wall time: 33 s


In [55]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,73,68
1,72,99


In [56]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.50      0.52      0.51       141
          1       0.59      0.58      0.59       171

avg / total       0.55      0.55      0.55       312



In [57]:
accuracy_score(yt, yp)

0.55128205128205132

In [58]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [59]:
ytp = model.predict(trainDataVecs)
ytt = train_w2v_data['openness_m']

In [60]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       0.99      1.00      1.00       360
          1       1.00      0.99      1.00       367

avg / total       1.00      1.00      1.00       727



In [61]:
print accuracy_score(ytt, ytp)

0.995873452545
